In [1]:
!pip install datasets evaluate sacrebleu transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 474.6/474.6 kB 8.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.4/81.4 kB 6.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 118.9/118.9 kB 10.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 68.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 19.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 26.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.5/212.5 kB 23.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 16.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.3/134.3 kB 19.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 27.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.5/114.5 kB 13.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━

In [2]:
from datasets import load_dataset

dataset = load_dataset("kde4", lang1="ar", lang2="en")
dataset

Generating train split: 0 examples [00:00, ? examples/s]

Dataset kde4 downloaded and prepared to /root/.cache/huggingface/datasets/kde4/ar-en-lang1=ar,lang2=en/0.0.0/243129fb2398d5b0b4f7f6831ab27ad84774b7ce374cf10f60f6e1ff331648ac. Subsequent calls will reuse this data.


  0%|          | 0/1 [00:00<?, ?it/s]

DatasetDict({
    train: Dataset({
        features: ['id', 'translation'],
        num_rows: 116239
    })
})

In [3]:
test_dataset = dataset['train'].train_test_split(test_size=.2)
val_dataset = test_dataset['test'].train_test_split(test_size=.5)
dataset['validation'] = val_dataset['train']
dataset['test'] = val_dataset['test']
dataset

DatasetDict({
    train: Dataset({
        features: ['id', 'translation'],
        num_rows: 116239
    })
    validation: Dataset({
        features: ['id', 'translation'],
        num_rows: 11624
    })
    test: Dataset({
        features: ['id', 'translation'],
        num_rows: 11624
    })
})

In [4]:
from transformers import TFAutoModel, AutoTokenizer, AutoConfig

checkpoint = 't5-base'

old_tokenizer = AutoTokenizer.from_pretrained(checkpoint)

/usr/local/lib/python3.10/dist-packages/transformers/models/t5/tokenization_t5_fast.py:155: FutureWarning: This tokenizer was incorrectly instantiated with a model max length of 512 which will be corrected in Transformers v5.
For now, this behavior is kept to avoid breaking backwards compatibility when padding/encoding with `truncation is True`.
- Be aware that you SHOULD NOT rely on t5-base automatically truncating your input to 512 when padding/encoding.
- If you want to encode/pad to sequences longer than 512 you can either instantiate this tokenizer with `model_max_length` or pass `max_length` when encoding/padding.
- To avoid this warning, please instantiate this tokenizer with `model_max_length` set to your preferred value.
  warnings.warn(


In [5]:
AutoConfig.from_pretrained(checkpoint)

T5Config {
  "_name_or_path": "t5-base",
  "architectures": [
    "T5ForConditionalGeneration"
  ],
  "d_ff": 3072,
  "d_kv": 64,
  "d_model": 768,
  "decoder_start_token_id": 0,
  "dense_act_fn": "relu",
  "dropout_rate": 0.1,
  "eos_token_id": 1,
  "feed_forward_proj": "relu",
  "initializer_factor": 1.0,
  "is_encoder_decoder": true,
  "is_gated_act": false,
  "layer_norm_epsilon": 1e-06,
  "model_type": "t5",
  "n_positions": 512,
  "num_decoder_layers": 12,
  "num_heads": 12,
  "num_layers": 12,
  "output_past": true,
  "pad_token_id": 0,
  "relative_attention_max_distance": 128,
  "relative_attention_num_buckets": 32,
  "task_specific_params": {
    "summarization": {
      "early_stopping": true,
      "length_penalty": 2.0,
      "max_length": 200,
      "min_length": 30,
      "no_repeat_ngram_size": 3,
      "num_beams": 4,
      "prefix": "summarize: "
    },
    "translation_en_to_de": {
      "early_stopping": true,
      "max_length": 300,
      "num_beams": 4,
      "pre

In [6]:
def create_inputs_labels(examples):
  inputs = [ex['ar'] for ex in examples['translation']]
  labels = [ex['en'] for ex in examples['translation']]

  return {'inputs': inputs, 'targets': labels}
          
cleaned_dataset = dataset.map(create_inputs_labels, batched=True, remove_columns=dataset['train'].column_names)
cleaned_dataset

Map:   0%|          | 0/116239 [00:00<?, ? examples/s]

Map:   0%|          | 0/11624 [00:00<?, ? examples/s]

Map:   0%|          | 0/11624 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['inputs', 'targets'],
        num_rows: 116239
    })
    validation: Dataset({
        features: ['inputs', 'targets'],
        num_rows: 11624
    })
    test: Dataset({
        features: ['inputs', 'targets'],
        num_rows: 11624
    })
})

In [7]:
tokenizer = old_tokenizer.train_new_from_iterator(cleaned_dataset['train']['inputs'] + cleaned_dataset['train']['targets'], 32128)

In [8]:
tokenizer.tokenize(dataset['train']['translation'][13333]['en'])

['▁<', '▁pattern', '▁>']

In [9]:
tokenizer.tokenize(dataset['train']['translation'][13333]['ar'])

['▁التشكيلة']

In [10]:
max_len = 128
def tokenize(examples):
  return tokenizer(examples['inputs'], text_target=examples['targets'], max_length=max_len, truncation=True)

In [11]:
tokenized_dataset = cleaned_dataset.map(tokenize, batched=True, remove_columns=cleaned_dataset['train'].column_names)
tokenized_dataset

Map:   0%|          | 0/116239 [00:00<?, ? examples/s]

Map:   0%|          | 0/11624 [00:00<?, ? examples/s]

Map:   0%|          | 0/11624 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 116239
    })
    validation: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 11624
    })
    test: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 11624
    })
})

In [27]:
config = {
    'batch_size': 16,
    'epochs': 7
}

In [13]:
from transformers import TFAutoModelForSeq2SeqLM

model = TFAutoModelForSeq2SeqLM.from_pretrained(checkpoint)

All model checkpoint layers were used when initializing TFT5ForConditionalGeneration.

All the layers of TFT5ForConditionalGeneration were initialized from the model checkpoint at t5-base.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFT5ForConditionalGeneration for predictions without further training.


In [14]:
from transformers import DataCollatorForSeq2Seq

datacollator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model=model, return_tensors='tf')

In [28]:
train_ds = tokenized_dataset['train'].to_tf_dataset(
                                          shuffle=True,
                                          batch_size=config['batch_size'], 
                                          collate_fn=datacollator,
                                          columns=tokenized_dataset['train'].column_names)
val_ds = tokenized_dataset['validation'].to_tf_dataset(
                                          shuffle=True,
                                          batch_size=config['batch_size'], 
                                          collate_fn=datacollator,
                                          columns=tokenized_dataset['validation'].column_names)
test_ds = tokenized_dataset['test'].to_tf_dataset(
                                          shuffle=True,
                                          batch_size=config['batch_size'], 
                                          collate_fn=datacollator,
                                          columns=tokenized_dataset['test'].column_names)

In [29]:
for i in train_ds.take(1):
  print(i)

{'input_ids': <tf.Tensor: shape=(16, 128), dtype=int64, numpy=
array([[  282,  1786,   116, ...,     0,     0,     0],
       [  302,  1317,   139, ...,     0,     0,     0],
       [ 8487,   346, 12326, ...,     0,     0,     0],
       ...,
       [  305,  7690,   676, ...,     0,     0,     0],
       [29116,   104, 30145, ...,     0,     0,     0],
       [23409,   184,  6086, ...,     0,     0,     0]])>, 'attention_mask': <tf.Tensor: shape=(16, 128), dtype=int64, numpy=
array([[1, 1, 1, ..., 0, 0, 0],
       [1, 1, 1, ..., 0, 0, 0],
       [1, 1, 1, ..., 0, 0, 0],
       ...,
       [1, 1, 1, ..., 0, 0, 0],
       [1, 1, 1, ..., 0, 0, 0],
       [1, 1, 1, ..., 0, 0, 0]])>, 'labels': <tf.Tensor: shape=(16, 20), dtype=int64, numpy=
array([[  190,   182,   626,   107,   530,   105,  3736,   113,  1305,
          402,   440,   967,  1316,   388,   141,  1305,   182,  1716,
          106,     1],
       [  430,  7203,   761, 13038,     1,  -100,  -100,  -100,  -100,
         -100,  -1

In [17]:
import evaluate

metric = evaluate.load("sacrebleu")

In [30]:
import numpy as np
import tensorflow as tf
from tqdm import tqdm

@tf.function(jit_compile=True)
def generate_with_xla(batch):
    return model.generate(
        input_ids=batch["input_ids"],
        attention_mask=batch["attention_mask"],
        max_new_tokens=128,
    )


def compute_metrics():
    all_preds = []
    all_labels = []

    for batch, labels in tqdm(test_ds):
        predictions = generate_with_xla(batch)
        decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
        labels = labels.numpy()
        labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
        decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)
        decoded_preds = [pred.strip() for pred in decoded_preds]
        decoded_labels = [[label.strip()] for label in decoded_labels]
        all_preds.extend(decoded_preds)
        all_labels.extend(decoded_labels)

    result = metric.compute(predictions=all_preds, references=all_labels)
    return {"bleu": result["score"]}

In [31]:
from transformers import create_optimizer
import tensorflow as tf

num_train_steps = len(train_ds) * config['epochs']

optimizer, schedule = create_optimizer(
    init_lr=5e-5,
    num_warmup_steps=0,
    num_train_steps=num_train_steps,
    weight_decay_rate=0.01,
)
model.compile(optimizer=optimizer)

No loss specified in compile() - the model's internal loss computation will be used as the loss. Don't panic - this is a common way to train TensorFlow models in Transformers! To disable this behaviour please pass a loss argument, or explicitly pass `loss=None` if you do not want your model to compute a loss.


In [32]:
model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=config['epochs']
)

Epoch 1/5
7265/7265 [==============================] - 2702s 367ms/step - loss: 6.6057 - val_loss: 5.9005
Epoch 2/5
7265/7265 [==============================] - 2656s 366ms/step - loss: 5.8834 - val_loss: 5.4414
Epoch 3/5
7265/7265 [==============================] - 2649s 365ms/step - loss: 5.5494 - val_loss: 5.1623
Epoch 4/5
7265/7265 [==============================] - 2597s 357ms/step - loss: 5.3592 - val_loss: 5.0223
Epoch 5/5
7265/7265 [==============================] - 2597s 357ms/step - loss: 5.2607 - val_loss: 4.9672
